# Session 8 - Interpolation

If you haven't already installed the packages `xlrd` and `scipy`, you can install them by opening the Anaconda Prompt and type:

* `pip install xlrd`

* `pip install scipy`

# Exercise 1

This exercise is mostly about reading and understanding code written by others. Which is often just as important as being able to write it oneself.

## Exercise 1.1
Read through the code given in the scipt below and try to understand what it does and how it does it.

Copy the script tot he editor and run it. 

Add print statements if you are unsure about how a certain varaible looks at any point. Remember you can print the first five rows of a dataframe with `df.head()`.    

The script reads two Excel files. One limitation of this is that they cannot be read while they are open. If you want to inspect these files while running the script, create a copy. 

## Exercise 1.2
The variable `settlements_interpolated` is a numpy array (you can see this by `type(settlements_interpolated)` => `<class 'numpy.ndarray'>`).

The last part of the code creates a figure object and an axis object which enables 3D plots.

Continue the plotting code to add:

* 3D scatter points of the known points $(x_{known}, y_{known}, w_{known})$

* 3D scatter points of the interpoalted settlements in every base slab node $(x_{nodes}, y_{nodes}, w_{nodes})$

In the above $w$ denotes the settlements.

## Exercise 1.3
All the settlement values in the base slab are in this example to be transferred to and applied in a Sofistik calculation.

To this end, we can create a text file with the exact syntax that the Sofistik input language accepts (this language is called CADINP and is somewhat similar to IBDAS's input language). The file has to be a `.dat`-file.

To write data to a file we can use something called **context managers**. Basically, it allows us to open a file and write to it. See code snippet below:

---
~~~python
# Use a context manager to open and write ('w') to file
with open('file_name.dat', 'w') as file:
    
    # The file can from here on out be referred to as file
    file.write("This text will be written inside 'file_name.dat'")
~~~
---

By using the concept the file is automatically closed after our indented block is terminated. It also creates the file in case it doesn't already exist. 

The Sofistik input file we want has the format:

~~~html
+PROG SOFILOAD 

LC 25 type 'P' fact 1.0 facd 0.0 titl 'LT settlement all nodes'

  POIN NODE 'insert first node number' WIDE 0 TYPE WZZ 'insert first settlement value'
  ... 'one line per base slab node' ... 
  POIN NODE 'insert last node number' WIDE 0 TYPE WZZ 'insert last settlement value'
  
END
~~~

The indented block should print all the node/settlement pairs. The three non-indented lines should only appear once. The output file should look like the file `settlement_field_generated_code_example.dat` in the folder.


### The script

~~~python

import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# Set name of Excel file to read
file_known = 'known_sections_plaxis.xlsx'

# Set name of sheet to read from Excel file
sheet_known = 'known_sections_plaxis'

# Read data from Excel sheet into a dataframe
df = pd.read_excel(file_known, sheet_name=sheet_known, skiprows=7)

# Extract columns whose names starts with a 'Y' into new dataframe of Y-coordinates
df_y = df[df.columns[df.columns.str.startswith('Y')]]

# Extract columns whose names starts with 'Z' into new dataframe of settlements
df_settlements_known = df[df.columns[df.columns.str.startswith('Z')]]

# Flatten dataframe values into 1D array
y_known = df_y.values.flatten()
settlements_known = df_settlements_known.values.flatten()

# Extract known x-values
x_known = df['X']

# Create X-array by repeating itself as many times as there are Y-columns
# This will create matching(x, y)-points between arrays x and y
x_known = np.repeat(x_known, len(df_y.columns))

# Set names and read Excel file with base slab nodes
file_nodes = 'base_slab_nodes.xlsx'
sheet_nodes = 'XLSX-Export'
df_nodes = pd.read_excel(file_nodes, sheet_name=sheet_nodes)

# Extract x- and y-coordinates of nodes
x_nodes = df_nodes['X [m]']
y_nodes = df_nodes['Y [m]']

# Extract node numbers
node_no = df_nodes['NR']

# Mirror known y-values and add corresponding x-values and settlements
x_known = np.append(x_known, x_known)
y_known = np.append(y_known, -y_known)
settlements_known = np.append(settlements_known, settlements_known)

# Arrange known (x, y) points to fit input for interpolation
xy_known = np.array(list(zip(x_known, y_known)))

# Perform interpolation calculation
settlements_interpolated = griddata(xy_known, settlements_known, (x_nodes, y_nodes), method='cubic')


####################
### Exercise 1.2 ###
####################
# Create figure object
fig = plt.figure()

# Create axis object for 3D plot
ax = fig.add_subplot(111, projection='3d')

# Plot known settlement points as 3D scatter plot (ax.scatter(...))
    # <Put plotting code here!>

# Plot interpolated field as 3D scatter plot
    # <Put plotting code here!>

# Show figure
    # <Put plotting code here!>


####################
### Exercise 1.3 ###
####################
# Write Sofistik input code to .dat-file for applying settlements as imposed displacements
    # <Put plotting code here!>
   
  

~~~